In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import numpy as np
import pandas as pd
from bayesian_testing.experiments import BinaryDataTest, DeltaLognormalDataTest

In [2]:
df = pd.read_csv("data/session_data.csv")

In [3]:
# example session data - each row represent one session
len(df)
df.head()

94500

,conversion,date,revenue,source,variant
0,0,2021-08-07,0.000000,desktop,B
1,1,2021-08-05,7.241015,desktop,C
2,0,2021-08-06,0.000000,desktop,A
3,0,2021-08-05,0.000000,desktop,C
4,0,2021-08-03,0.000000,desktop,A


In [4]:
# summary statistics per variant

summary = df.groupby('variant')[['variant', 'conversion', 'revenue']]\
            .agg({'variant': 'count', 'conversion': 'sum','revenue': 'sum'})\
            .rename(columns = {'variant': 'sessions', 'conversion': 'conversions'})

summary['conversion_rate'] = summary['conversions'] / summary['sessions']
summary['revenue_per_session'] = summary['revenue'] / summary['sessions']
summary['revenue_per_converted_sessions'] = summary['revenue'] / summary['conversions']

summary

,sessions,conversions,revenue,conversion_rate,revenue_per_session,revenue_per_converted_sessions
variant,,,,,,
A,31500,1580,30830.025613,0.050159,0.978731,19.512674
B,32000,1700,35203.216888,0.053125,1.100101,20.707775
C,31000,1550,37259.563364,0.050000,1.201921,24.038428


# Conversion Test

In [5]:
conv_test = BinaryDataTest()

In [6]:
variant_A = df['conversion'][(df.variant == 'A')].values
variant_B = df['conversion'][(df.variant == 'B')].values
variant_C = df['conversion'][(df.variant == 'C')].values

variant_A
max(variant_A)

array([0, 0, 0, ..., 0, 0, 0])

1

In [7]:
conv_test.add_variant_data("A", variant_A)

In [8]:
conv_test.add_variant_data("B", variant_B)

In [9]:
conv_test.add_variant_data_agg("C", len(variant_C), sum(variant_C))

In [10]:
conv_test.data
conv_test.variant_names
conv_test.totals
conv_test.positives

{'A': {'totals': 31500, 'positives': 1580, 'a_prior': 0.5, 'b_prior': 0.5},
 'B': {'totals': 32000, 'positives': 1700, 'a_prior': 0.5, 'b_prior': 0.5},
 'C': {'totals': 31000, 'positives': 1550, 'a_prior': 0.5, 'b_prior': 0.5}}

['A', 'B', 'C']

[31500, 32000, 31000]

[1580, 1700, 1550]

In [11]:
conv_test.probabs_of_being_best(sim_count = 20000, seed=52)

{'A': 0.04185, 'B': 0.92235, 'C': 0.0358}

In [12]:
conv_test.expected_loss(sim_count = 20000, seed=52)

{'A': 0.0030138, 'B': 6.06e-05, 'C': 0.0031649}

In [13]:
conv_test.evaluate()

[{'variant': 'A',
  'totals': 31500,
  'positives': 1580,
  'positive_rate': 0.05016,
  'prob_being_best': 0.03885,
  'expected_loss': 0.0030295},
 {'variant': 'B',
  'totals': 32000,
  'positives': 1700,
  'positive_rate': 0.05312,
  'prob_being_best': 0.92935,
  'expected_loss': 4.97e-05},
 {'variant': 'C',
  'totals': 31000,
  'positives': 1550,
  'positive_rate': 0.05,
  'prob_being_best': 0.0318,
  'expected_loss': 0.0031817}]

In [14]:
#reversed_probabilities (for case when "best = min")
conv_test.probabs_of_being_best(sim_count = 20000, seed=52, min_is_best=True)
conv_test.expected_loss(sim_count = 20000, seed=52, min_is_best=True)

{'A': 0.4594, 'B': 0.00925, 'C': 0.53135}

{'A': 0.000781, 'B': 0.0037342, 'C': 0.0006299}

# Revenue Test

In [15]:
rev_test = DeltaLognormalDataTest()

In [16]:
variant_A = df['revenue'][(df.variant == 'A')].values
variant_B = df['revenue'][(df.variant == 'B')].values
variant_C = df['revenue'][(df.variant == 'C')].values

variant_A
max(variant_A)

array([0., 0., 0., ..., 0., 0., 0.])

318.85100060418284

In [17]:
rev_test.add_variant_data("A", variant_A)
rev_test.add_variant_data("B", variant_B)

In [18]:
# adding variant using aggregated data
rev_test.add_variant_data_agg(
    name="C",
    totals=len(variant_C),
    positives=sum(x > 0 for x in variant_C),
    sum_values=sum(variant_C),
    sum_logs=sum([np.log(x) for x in variant_C if x > 0]),
    sum_logs_2=sum([np.square(np.log(x)) for x in variant_C if x > 0]),
)

In [19]:
# rev_test.data
rev_test.variant_names
rev_test.totals
rev_test.positives
rev_test.sum_values
rev_test.sum_logs
rev_test.sum_logs_2

['A', 'B', 'C']

[31500, 32000, 31000]

[1580, 1700, 1550]

[30830.02561293968, 35203.216888281, 37259.56336401027]

[3831.806394737818, 4211.729867679853, 4055.9652348481686]

[11029.923165846512, 12259.518683969136, 12357.91186291399]

In [20]:
rev_test.probabs_of_being_best()

{'A': 0.0003, 'B': 0.0336, 'C': 0.9661}

In [21]:
rev_test.expected_loss()

{'A': 0.2217132, 'B': 0.1213773, 'C': 0.0008132}

In [22]:
rev_test.evaluate()

[{'variant': 'A',
  'totals': 31500,
  'positives': 1580,
  'sum_values': 30830.02561,
  'avg_values': 0.97873,
  'avg_positive_values': 19.51267,
  'prob_being_best': 0.00015,
  'expected_loss': 0.2198242},
 {'variant': 'B',
  'totals': 32000,
  'positives': 1700,
  'sum_values': 35203.21689,
  'avg_values': 1.1001,
  'avg_positive_values': 20.70777,
  'prob_being_best': 0.036,
  'expected_loss': 0.1195017},
 {'variant': 'C',
  'totals': 31000,
  'positives': 1550,
  'sum_values': 37259.56336,
  'avg_values': 1.20192,
  'avg_positive_values': 24.03843,
  'prob_being_best': 0.96385,
  'expected_loss': 0.0009141}]

# Probabilities in time

In [23]:
# using time data we can generate probabilities of being best over time (cumulatively)

dates = np.sort(df.date.unique())
pbbs_conv_in_time = []
pbbs_rev_in_time = []
for i in dates:
    variant_A_conv = df[(df.variant == 'A') & (df.date <= i)]['conversion'].values
    variant_B_conv = df[(df.variant == 'B') & (df.date <= i)]['conversion'].values
    variant_C_conv = df[(df.variant == 'C') & (df.date <= i)]['conversion'].values
    
    variant_A_rev = df[(df.variant == 'A') & (df.date <= i)]['revenue'].values
    variant_B_rev = df[(df.variant == 'B') & (df.date <= i)]['revenue'].values
    variant_C_rev = df[(df.variant == 'C') & (df.date <= i)]['revenue'].values

    conv_test = BinaryDataTest()
    conv_test.add_variant_data("A", variant_A_conv)
    conv_test.add_variant_data("B", variant_B_conv)
    conv_test.add_variant_data("C", variant_C_conv)
    
    rev_test = DeltaLognormalDataTest()
    rev_test.add_variant_data("A", variant_A_rev)
    rev_test.add_variant_data("B", variant_B_rev)
    rev_test.add_variant_data("C", variant_C_rev)
    
    pbbs_conv_in_time.append({i: conv_test.probabs_of_being_best()})
    pbbs_rev_in_time.append({i: rev_test.probabs_of_being_best()})

In [24]:
pbbs_conv_in_time

[{'2021-08-01': {'A': 0.6985, 'B': 0.1836, 'C': 0.1179}},
 {'2021-08-02': {'A': 0.3602, 'B': 0.2286, 'C': 0.4112}},
 {'2021-08-03': {'A': 0.19425, 'B': 0.6478, 'C': 0.15795}},
 {'2021-08-04': {'A': 0.159, 'B': 0.74195, 'C': 0.09905}},
 {'2021-08-05': {'A': 0.02995, 'B': 0.87575, 'C': 0.0943}},
 {'2021-08-06': {'A': 0.01425, 'B': 0.9721, 'C': 0.01365}},
 {'2021-08-07': {'A': 0.0405, 'B': 0.92695, 'C': 0.03255}}]

In [25]:
pbbs_rev_in_time

[{'2021-08-01': {'A': 0.0241, 'B': 0.26735, 'C': 0.70855}},
 {'2021-08-02': {'A': 0.05465, 'B': 0.2137, 'C': 0.73165}},
 {'2021-08-03': {'A': 0.0653, 'B': 0.2081, 'C': 0.7266}},
 {'2021-08-04': {'A': 0.02145, 'B': 0.1797, 'C': 0.79885}},
 {'2021-08-05': {'A': 0.00035, 'B': 0.0455, 'C': 0.95415}},
 {'2021-08-06': {'A': 0.00035, 'B': 0.0499, 'C': 0.94975}},
 {'2021-08-07': {'A': 0.0002, 'B': 0.03455, 'C': 0.96525}}]